In [83]:
import mip

In [84]:
jobs = range(4)
p = [6, 4, 5, 8] # assume all positive
d = [8, 4, 12, 16] # assume all positive

In [85]:
model = mip.Model()

In [86]:
x = [[model.add_var(name=f"X{i}{j}", var_type=mip.BINARY) for i in jobs] for j in jobs] # 1 iff job i after job j
t = [model.add_var(name=f"T{i}", lb=0, var_type=mip.CONTINUOUS) for i in jobs] # time at which job i is completed
k = [model.add_var(name=f"K{i}", lb=0, var_type=mip.CONTINUOUS) for i in jobs] # max(t, 0)
# t and k must be integer, no need to enforce because p and d are all integer

M = sum(p) + sum(d)

In [87]:
model.objective = mip.minimize(mip.xsum([k[i] for i in jobs]))

In [88]:
for i in jobs:
    model.add_constr(t[i] >= p[i])

for i in jobs:
    for j in jobs:
        if i == j: # non ti interessa il caso in cui hai lo stesso job --> la matrice è già antitrasposta, non devi forzarla
            continue
        model.add_constr(t[i] >= t[j] + p[i] - M * (1 - x[i][j]))
        model.add_constr(t[j] >= t[i] + p[j] - M * x[i][j])

for i in jobs:
    model.add_constr(k[i] >= t[i] - d[i])

In [89]:
status = model.optimize()

Cgl0004I processed model has 16 rows, 20 columns (12 integer (12 of which binary)) and 44 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 1.463%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0038I Initial state - 12 integers unsatisfied sum - 1.09524
Cbc0038I Pass   1: suminf.    1.09524 (8) obj. 11 iterations 6
Cbc0038I Pass   2: suminf.    0.36508 (2) obj. 29 iterations 15
Cbc0038I Solution found of 29
Cbc0038I Relaxing continuous gives 29
Cbc0038I Before mini branch and bound, 0 integers at bound fixed and 2 continuous
Cbc0038I Full problem 16 rows 20 columns, reduced to 9 rows 12 columns
Cbc0038I Mini branch and bound improved solution from 29 to 28 (0.01 seconds)
Cbc0038I Round again with cutoff of 24.3001
Cbc0038I Pass   3: suminf.    1.09524 (8) obj. 11 iterations 0
Cbc0038I Pass   4: suminf.    0.36508 (3) obj. 24.3001 iterations 14
Cbc0038I Pass   5: suminf.    0.19048 (1) obj. 21 iterations 8
Cbc0038I Solution found of 21
Cbc0038I Re

In [90]:
status == mip.OptimizationStatus.OPTIMAL

True

In [91]:
model.objective.x

12.0

In [92]:
for var in model.vars: # print the time at which each job will be completed
    if var.name[0] == "T":
        print(var.name, "\t=\t", var.x)

T0 	=	 10.0
T1 	=	 4.0
T2 	=	 15.0
T3 	=	 23.0
